# Introduction

<div class="alert alert-block alert-success">
<b>Ce notebook permet d'effectuer les analyses suivantes:</b>
</div>

- Calculer des courbes de modulation adimensionnelle et "semainisée"
- Comparer jour de semaine / jour de weekend
- Comparer les jours de semaine entre eux
- Comparer les mois de l'année

<div class="alert alert-block alert-success">
<b>Format de la donnée en entrée: </b> fichier pickle produit par le notebook de traitement de la donnée brute
</div>

<div class="alert alert-block alert-warning">
<b>Possibles améliorations: </b> Il est possible d'ajouter les informations statistiques sur les courbes de modulation (Q1 et Q3)
</div>

<div class="alert alert-block alert-warning">
<b>Format de la donnée en entrée: </b> Il est possible de modifier le notebook afin de spécifier des fichiers CSV en entrée (donnée brute et fuites). Cependant les fichiers pickle sont plus efficaces et plus rapides à utiliser.
</div>

# Librairies

In [1]:
import pandas as pd
import numpy as np
import pickle
import ntpath
import seaborn as sns
from datetime import datetime as dt
from datetime import timedelta as td
import matplotlib.pyplot as plt

# Interaction
import ipywidgets as widgets
from ipywidgets import interact
from ipywidgets import GridspecLayout, Layout
from IPython.display import display, Markdown, clear_output
from ipyfilechooser import FileChooser

from IPython.core.display import display, HTML
display(HTML("<style>.container {width:75% !important;margin-left:25%;}</style>"))

# Entrée utilisateur

<div class="alert alert-block alert-warning">
<b>Possible amélioration: </b> utiliser un widget "File selection" pour rendre le processus plus simple d'utilisation 
</div>

In [7]:
# Fichier Paramètres
paramSheetPath = r"C:\Users\alde\OneDrive - DHI\Documents\DHI Projects\21803508_SWDE\06 Tools\Parameters\parametres_modulotool.xlsx"
paramsdf = pd.read_excel(paramSheetPath)
paramsdf = paramsdf[paramsdf.Active]

datasets=[]
for idx, row in paramsdf.iterrows():
    if row['Path_clean'].endswith('.pkl'):
        with open(row['Path_clean'], 'rb') as f:
            data = pickle.load(f)
        datasets.append({'Nom':row['Nom'],
                         'Path_raw':row['Path_raw'],
                         'Path_clean':row['Path_clean'],
                         'Path_leaks':row['Path_leaks'],
                         'Path_report':row['Path_report'],
                         'Data_clean':data})
    elif row['Path_clean'].endswith('.csv'):
        data = pd.read_csv(row['Path_clean'], sep=';', index_col='DATE', parse_dates=True)
        datasets.append({'Nom':row['Nom'],
                         'Path_raw':row['Path_raw'],
                         'Path_clean':row['Path_clean'],
                         'Path_leaks':row['Path_leaks'],
                         'Path_report':row['Path_report'],
                         'Data_clean':data})

if False:
    # Lecture des jeux de données
    datasets = {}
    for rowIdx, row in params.iterrows():
        # Récupérer le nom du jeu de donnée (sinon defaut: nom du fichier)
        if row['Nom'] != '':
            datasetName = row['Nom']
        else:
            datasetName = ntpath.basename(row['Path'])

        # Récupérer la donnée suivant le type de fichier
        if row['Path'].endswith('.xlsx'):
            data = pd.read_excel(row['Path'], sheet_name=row['Column'])
        elif row['Path'].endswith('.pkl'):
            with open(row['Path'], 'rb') as f:
                allData = pickle.load(f)
                data_w_leaks = allData['data_w_leaks']
                data = allData['data_wout_leaks']
                leaks = allData['leaks']
                del allData
        elif row['Path'].endswith('.csv'):
            print('CSV files not supported yet')
            continue

        # Enregistrer dans le dictionnaire de jeux de donnée
        datasets[datasetName] = data

print('DONE')
print('Données chargées:')
print('\t- ' + '\n\t- '.join([d['Nom'] for d in datasets]))

DONE
Données chargées:
	- Malmedy
	- Charleroi


# Fonctions personnalisées et options graphiques

In [18]:
# Quelques préférences graphiques pour la suite
figSize = (9.5,5)
figSizeTall = (9.5,8)
fontsizeFigureTitle = 18
fontsizeAxisTitle = 12
fontsizeAxisLabel = 10

# Liste des jours de la semaine pour représenatation graphique ci-dessous (doit être en anglais)
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Pour rendre les figures interactives + quelques options pour graphismes
%matplotlib notebook
layout = widgets.Layout(width='auto')
layoutDD = widgets.Layout(width='500px')
sns.set_style("whitegrid")
sns.set_palette("Dark2")

# Fonction 
def WeekdayWeekend(day):
    if day in ['Saturday', 'Sunday']:
        return 'weekend'
    else:
        return 'weekday'

# Simple function to convert date to string in hard-coded format
def date2str(date):
    return dt.strftime(date, '%d-%m-%Y')

def FormatAxTitle(title):
    splitTitle = title.split(' ')
    stringBreakpoint = int(len(splitTitle)/2)
    formattedTitle = ' '.join(splitTitle[:stringBreakpoint]) + '\n' + ' '.join(splitTitle[stringBreakpoint:])
    return formattedTitle

def CreateCopyButton(description, button_style):
    return widgets.Button(description=description, button_style=button_style, icon='check', layout=Layout(height='auto', width='auto',color='white'))

def CopyDataFrameToClipboard(df, excel=True):
    df.to_clipboard(excel=excel)
    
def PrepareDataframe(df):
    # Ajout des colonnes mois / jour de  semaine ou weekend / jour de la semaine à la dataframe
    df['month_name'] = df.index.month_name()
    df['weekday_name'] = df.index.day_name()
    df['weekday'] = df['weekday_name'].apply(lambda x: WeekdayWeekend(x))

    # Normalisation des données par la moyenne
    dfNorm = df.copy(deep=True)
    for i in [i for i in df.columns if i not in ['month_name', 'weekday_name', 'weekday']]:
        dfNorm[i] = dfNorm[i]/dfNorm[i].mean()
    
    return df, dfNorm

# Choix du jeu de données

In [10]:
datasetsOptions = [i['Nom'] for i in datasets]

selectDatasetWidget = widgets.Dropdown(
    options=datasetsOptions,
    value=datasetsOptions[0],
    description='Input dataset:',
    disabled=False,)

if False:
    @interact
    def SelectDataset(dsName=selectDatasetWidget):
        df = next(dikt['Data_raw'] for dikt in datasets if dikt['Nom']==dsName)
        dataCols = df.columns
        display(dataCols)
        return df

###### LAYOUT #######
# Create elements to display
out = widgets.Output()
info = widgets.Label('Choose a dataset')
ApproveButton = CreateCopyButton('Approve', 'success')

# Arrange the elements in a grid
grid_data = GridspecLayout(2, 5)
grid_data[0,1] = selectDatasetWidget
grid_data[0,3] = ApproveButton
grid_data[1,:] = out

# Display
display(grid_data)

df, dataCols, ds = None, None, None

@interact
def SelectDataset():
    out.clear_output()

    # Event listener on the submit button: function to execute when clicked
    @ApproveButton.on_click
    def react_on_click(b):
        out.clear_output()
        global df, dataCols, ds
        df = next(dikt['Data_clean']['data_wout_leaks'] for dikt in datasets if dikt['Nom']==selectDatasetWidget.value)
        dataCols = df.columns
        ds = next(dikt for dikt in datasets if dikt['Nom']==selectDatasetWidget.value)
        with out:
            print('Selected dataset: {}'.format(selectDatasetWidget.value))

# Maintenant on peut définir la dataframe pour stocker des informations sur le nettoyage
#outdf = pd.DataFrame(index=dataCols, columns=['Discharge (base)','Discharge (leaks)', 'Deleted datapoints', 'Deleted datapoints percentage'])


GridspecLayout(children=(Dropdown(description='Input dataset:', layout=Layout(grid_area='widget001'), options=…

interactive(children=(Output(),), _dom_classes=('widget-interact',))

# Import des données

In [12]:
# Ancienne méthode dans le cas d'un seul pickle file
if False:
    # Ouvrir le fichier pickle et extraire les données
    with open(dataPickleFilePath, 'rb') as f:
        allData = pickle.load(f)
        data_w_leaks = allData['data_w_leaks']
        data = allData['data_wout_leaks']
        leaks = allData['leaks']
        del allData

    # On va ajouter des colonnes après.
    # Ici on garde une trace des colonnes initiales
    dataCols = data.columns

    # Ajout des colonnes mois / jour de  semaine ou weekend / jour de la semaine à la dataframe
    data['month_name'] = data.index.month_name()
    data['weekday_name'] = data.index.day_name()
    data['weekday'] = data['weekday_name'].apply(lambda x: WeekdayWeekend(x))

    # Normalisation des données par la moyenne
    dataNorm = data.copy(deep=True)
    for i in dataCols:
        dataNorm[i] = dataNorm[i]/dataNorm[i].mean()

    # Liste des jours de la semaine pour représenatation graphique ci-dessous (doit être en anglais)
    weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']


In [21]:
import matplotlib
matplotlib.get_backend()

'nbAgg'

# Visualisation

## Courbes de modulation - jours de la semaine

In [19]:
# Choisir le fichier d'entrée à partir de la liste renseignée dans le tableur de paramètres
if False:
    fileOptions = list(datasets.keys()) #.Nom.to_list()
    selectFileWidget = widgets.Dropdown(
        options=fileOptions,
        value=fileOptions[0],
        description='Input file:',
        disabled=False,)


if False:
    @interact
    def SelectInputFile(f=selectFileWidget):
        data, dataNorm = PrepareDataframe(datasets[f])
        dataCols = [i for i in data.columns if i not in ['month_name', 'weekday_name', 'weekday']]
if True:
    data, dataNorm = PrepareDataframe(df)
    # Choisir la station à partir de ce qui est disponible dans la dataframe
    selectStationWidget = widgets.Dropdown(
        options=sorted(dataCols),
        value=sorted(dataCols)[0],
        description='Station',
        layout=layoutDD,
        disabled=False)

    @interact
    def SelectStation(station=selectStationWidget):
        plt.close('all')
        stationData_t = data[[station, 'weekday', 'weekday_name']]
        stationDataNorm_t = dataNorm[[station, 'weekday', 'weekday_name']]

        # Déterminer les dates disponibles pour la sation choisie
        dateOptions = [(date.strftime(' %d %b %Y '), date) for date in stationData_t.index.to_pydatetime()]
        index = (0, len(dateOptions)-1)

        selectDates = widgets.SelectionRangeSlider(
                options=dateOptions,
                index=index,
                description='Dates',
                orientation='horizontal',
                layout={'width': '600px'},
                continuous_update=False,)

        @interact
        def ShowSlider(d=selectDates):
            plt.close('all')
            # Analyser sur la période sélectionnée
            start = d[0]
            end = d[-1]
            stationData = stationData_t[(stationData_t.index>=start) & (stationData_t.index<=end)]
            stationDataNorm = stationDataNorm_t[(stationDataNorm_t.index>=start) & (stationDataNorm_t.index<=end)]

            # Initialiser la figure
            fig = plt.figure(figsize=figSizeTall)
            gs = fig.add_gridspec(3,2)
            fig.set_figheight(gs.nrows*4)
            fig.suptitle(station+'\n', fontsize=fontsizeFigureTitle, fontweight='bold')

            # 0. Plot débit journaliers
            if True:
                axis = fig.add_subplot(gs[0, :])
                stationDataDaily = stationData.resample('D').mean().mul(24)
                stationDataDaily.plot(ax=axis, legend=False, linewidth=1, color='tab:blue')
                axis.set_xlabel('')
                axis.set_ylabel('Débit journalier moyen [m$^3$/j]', fontweight='bold')

            # 1. Analyse sur les weekdays / weekends
            if True:
                axis = fig.add_subplot(gs[1, 0])
                axis.set_title(FormatAxTitle('Comparaison des courbes de modulation jour de semaine / jour de weekend'), fontsize=fontsizeAxisTitle)
                for day in stationDataNorm['weekday'].unique():
                    dayDf = stationDataNorm[station][stationDataNorm['weekday']==day]
                    dayDfToPlot = dayDf.groupby(dayDf.index.hour).median()
                    dayDfToPlot.plot(ax=axis, label=day)
                axis.plot([0,24], [1,1], '--', color='tab:grey', linewidth=2)
                axis.legend(bbox_to_anchor=(0.33, 0, .67, 0), loc="lower left", ncol=4, mode='expand')
                axis.set_xlabel('Heure de la journée', fontweight='bold')
                axis.set_ylabel('Débit normalisé', fontweight='bold')
                axis.set_xlim(-1, 24)
                axis.set_xticks([i for i in range(0,25,4)])
                axis.set_xticklabels([str(i)+'h' for i in axis.get_xticks()])


            # 2. Analyse sur chaque jour de la semaine
            if True:
                axis = fig.add_subplot(gs[1, 1])
                axis.set_title(FormatAxTitle('Comparaison des courbes de modulation par jour de la semaine'), fontsize=fontsizeAxisTitle)
                for day in weekdays: #stationData['weekday_name'].unique():
                    dayDf = stationDataNorm[station][stationDataNorm['weekday_name']==day]
                    dayDfToPlot = dayDf.groupby(dayDf.index.hour).median()
                    dayDfToPlot.plot(ax=axis, label=day)
                axis.plot([0,24], [1,1], '--', color='tab:grey', linewidth=2)
                axis.legend(bbox_to_anchor=(0.33, 0, .67, 0), loc="lower left", ncol=2, mode='expand')
                axis.set_xlabel('Heure de la journée', fontweight='bold')
                axis.set_ylabel('Débit normalisé', fontweight='bold')
                axis.set_xlim(-1, 24)
                axis.set_xticks([i for i in range(0,25,4)])
                axis.set_xticklabels([str(i)+'h' for i in axis.get_xticks()])


            # 3. percentiles (analyse Alicia modifiée)
            axis = fig.add_subplot(gs[2, :])
            axis2 = axis.twinx()
            # Histogramme (bar)
            nbins=100
            stationDataDaily.hist(ax=axis, bins=nbins, density=True, color='tab:blue', linewidth=0, grid=False)
            # Histogramme (cumulé)
            #stationData.resample('D').mean().mul(24).hist(ax=axis2, bins=nbins, cumulative=1, density=1, histtype='step', color='tab:orange', linewidth=1, grid=False)
            npNbins = np.arange(np.floor(stationDataDaily.min().values), np.ceil(stationDataDaily.max().values))
            vals, bins = np.histogram(stationDataDaily, bins=npNbins, density=True)
            x = [(bins[idx]+bins[idx+1])/2 for idx in range(npNbins.shape[0]-1)]
            y = np.cumsum(vals)
            axis2.plot(x, y, color='tab:red', zorder=0)
            # Ajout des percentiles 50 et 95
            for quant in [.5, .95]:
                quant_val = np.nanquantile(stationDataDaily, quant)
                quant_superscript = ''
                for letter in str(int(quant*100)):
                    quant_superscript = quant_superscript + '$_{}$'.format(letter)
                axis2.plot([quant_val, quant_val], [0, quant], '--', color='tab:red')
                axis2.text(quant_val, (quant/2), ' q{} = {:.0f}m$^3$/j'.format(quant_superscript, quant_val), color='tab:red')
            #axis2.scatter(np.nanquantile(stationDataDaily, [.5, .95]), [.5, .95], marker='o', color='tab:red')

            # Formattage
            axis.set_title(''), axis2.set_title('')
            axis.set_yticklabels([str(round(100*i,2))+'%' for i in axis.get_yticks()])
            axis.set_xlabel('Débit journalier (m$^3$/j)', fontweight='bold')
            axis.set_ylabel('Fréquence', fontweight='bold')
            axis2.set_ylabel('Fréquence cumulée', color='tab:red', fontweight='bold')
            axis2.set_ylim(0, 1.05)
            axis2.grid(False)

            # Réduire les marges
            plt.tight_layout()

interactive(children=(Dropdown(description='Station', layout=Layout(width='500px'), options=('AEROPOLE_CE_CM_F…

## Courbes de modulation - mois de l'année

In [43]:
# Choisir le fichier d'entrée à partir de la liste renseignée dans le tableur de paramètres
fileOptions = list(datasets.keys()) #.Nom.to_list()
selectFileWidget = widgets.Dropdown(
    options=fileOptions,
    value=fileOptions[0],
    description='Input file:',
    disabled=False,)



@interact
def SelectInputFile(f=selectFileWidget):
    data, dataNorm = PrepareDataframe(datasets[f])
    dataCols = [i for i in data.columns if i not in ['month_name', 'weekday_name', 'weekday']]

    # Choisir la station à partir de ce qui est disponible dans la dataframe
    selectStationWidget = widgets.RadioButtons(
        options=dataCols,
        value=dataCols[0],
        description='Station',
        layout=layout,
        disabled=False)

    @interact
    def SelectStation(station=selectStationWidget):
        plt.close('all')

        stationData_t = data[[station, 'month_name']]
        stationDataNorm_t = dataNorm[[station, 'month_name']]

        # Some setup and the interactive widget
        dateOptions = [(date.strftime(' %d %b %Y '), date) for date in stationData_t.index.to_pydatetime()]
        index = (0, len(dateOptions)-1)

        selectDates = widgets.SelectionRangeSlider(
                options=dateOptions,
                index=index,
                description='Dates',
                orientation='horizontal',
                layout={'width': '500px'},
                continuous_update=False,)

        @interact
        def ShowSlider(d=selectDates):

            start = d[0]
            end = d[-1]

            stationData = stationData_t[(stationData_t.index>=start) & (stationData_t.index<=end)]
            stationDataNorm = stationDataNorm_t[(stationDataNorm_t.index>=start) & (stationDataNorm_t.index<=end)]

            fig, ax = plt.subplots(2,1, figsize=(9.5,8))
            fig.suptitle(station+'\n', fontsize=fontsizeFigureTitle, fontweight='bold')

            # 1. Boxplot / violinplot avec les données au mois
            axis = ax[0]
            axis.set_title('Débit journalier en fonction du mois de l\'année', fontsize=fontsizeAxisTitle)
            stationDataDaily = stationData.resample('D').mean().mul(24)
            stationDataDaily['month_name'] = stationDataDaily.index.month_name()
            #sns.boxplot(data=stationDataDaily, x='month_name', y=station, ax=axis)
            sns.violinplot(data=stationDataDaily, x='month_name', y=station, ax=axis, inner='quartile', sat=.6, alpha=.5, bw=.5)
            axis.set_xlabel('Mois de l\'année', fontweight='bold')
            axis.set_xticklabels(axis.get_xticklabels(), rotation=45, ha='right')
            axis.set_ylabel('Débit journalier [m$^3$/j]', fontweight='bold')


            # 2. Courbes de modulation par mois de l'année
            axis = ax[1]
            axis.set_title('Comparaison des courbes de modulation par mois de l\'année', fontsize=fontsizeAxisTitle)
            for month in stationDataNorm['month_name'].unique():
                dayDf = stationDataNorm[station][stationDataNorm['month_name']==month]
                dayDfToPlot = dayDf.groupby(dayDf.index.hour).mean()
                dayDfToPlot.plot(ax=axis, label=month)
            axis.plot([0,24], [1,1], '--', color='tab:grey', linewidth=2)
            #axis.legend()
            axis.set_xlabel('Heure de la journée', fontweight='bold')
            axis.set_ylabel('Débit normalisé', fontweight='bold')
            axis.set_xlim(-1, 24)
            axis.set_xticks([i for i in range(0,25,4)])
            axis.set_xticklabels([str(i)+'h' for i in axis.get_xticks()])
            axis.legend(bbox_to_anchor=(.33, 0, .67, 0), loc="lower right", mode="expand", ncol=6)

            # Réduire les marges
            plt.tight_layout()

interactive(children=(Dropdown(description='Input file:', options=('données SWDE',), value='données SWDE'), Ou…

## Analyse des fuites

In [44]:
# Choisir le fichier d'entrée à partir de la liste renseignée dans le tableur de paramètres
fileOptions = list(datasets.keys()) #.Nom.to_list()
selectFileWidget = widgets.Dropdown(
    options=fileOptions,
    value=fileOptions[0],
    description='Input file:',
    disabled=False,)



@interact
def SelectInputFile(f=selectFileWidget):
    data, dataNorm = PrepareDataframe(datasets[f])
    dataCols = [i for i in data.columns if i not in ['month_name', 'weekday_name', 'weekday']]

    # Choisir la station à partir de ce qui est disponible dans la dataframe
    selectStationWidget = widgets.RadioButtons(
        options=dataCols,
        value=dataCols[0],
        description='Station',
        layout=layout,
        disabled=False)

    @interact
    def SelectStation(station=selectStationWidget):
        plt.close('all')

        df_leaks_daily_min = leaks.loc[:,station].resample('W-MON').min()#.shift(periods=int((nightEnd-nightStart)/2), freq='H')
        meanLeaks = df_leaks_daily_min.mean()

        # Plot les fuites journalières
        fig, ax = plt.subplots(figsize=figSize)
        fig.suptitle(station)
        ax.plot(data_w_leaks.index, data_w_leaks[station], linewidth=.1, zorder=0, color='tab:blue',
                label='Mesure nettoyée')
        ax.plot(df_leaks_daily_min.index, df_leaks_daily_min, zorder=1, color='tab:red', linestyle='--',
                linewidth=1, label='Fuites journalières moyennes')
        xLims = data_w_leaks.index.min(), data_w_leaks.index.max()#ax.get_xlim()
        ax.plot([xLims[0], xLims[1]], [0,0], color='tab:grey')
        ax.plot([xLims[0], xLims[1]], [meanLeaks, meanLeaks], color='tab:red', linewidth=.5)
        ax.text(xLims[1]+td(days=1), meanLeaks, 'Moyenne: {:.1f}m$^3$/h'.format(meanLeaks), va='center', color='tab:red')
        ax.set_ylabel('Débit [m$^3$/h]')
        ax.legend(bbox_to_anchor=(0,1,1,0), ncol=3, fontsize=8.5)
        plt.tight_layout()
    

interactive(children=(Dropdown(description='Input file:', options=('données SWDE',), value='données SWDE'), Ou…

## Tableurs

### Courbes de modulation - weekend et weekday

In [14]:
# Choisir le fichier d'entrée à partir de la liste renseignée dans le tableur de paramètres
fileOptions = list(datasets.keys()) #.Nom.to_list()
selectFileWidget = widgets.Dropdown(
    options=fileOptions,
    value=fileOptions[0],
    description='Input file:',
    disabled=False,)



@interact
def SelectInputFile(f=selectFileWidget):
    data, dataNorm = PrepareDataframe(datasets[f])
    dataCols = [i for i in data.columns if i not in ['month_name', 'weekday_name', 'weekday']]

    # Create a list of the stations available
    selectStationWidget = widgets.RadioButtons(
        options=dataCols,
        value=dataCols[0],
        description='Station',
        layout=layout,
        disabled=False)


    ###### LAYOUT #######
    # Create elements to display
    out = widgets.Output()
    info = widgets.Label('Choose a station')
    CopyButton = CreateCopyButton('Copy to clipboard', 'success')

    # Arrange the elements in a grid
    grid_data = GridspecLayout(2, 5)
    grid_data[1,1] = info
    grid_data[1,3] = CopyButton
    grid_data[0,:] = out

    # Display
    display(grid_data)


    ###### INTERACTIONS #######
    @interact
    def SelectStation(station=selectStationWidget):
        out.clear_output()
        plt.close('all')
        #stationData = data[[station, 'weekday', 'weekday_name']]
        stationDataNorm = dataNorm[[station, 'weekday', 'weekday_name']]

        # 1. Analyse sur les weekdays / weekends
        toDisplay = pd.DataFrame()
        for day in stationDataNorm['weekday'].unique():
            dayDf = stationDataNorm[station][stationDataNorm['weekday']==day]
            dayDfToPlot = dayDf.groupby(dayDf.index.hour).median()
            toDisplay['{}_{}'.format(station, day)] = dayDfToPlot
            #dayDfToPlot.rename({station:'{}_{}'.format(station, day)}, inplace=True)
            #print(dayDfToPlot.columns)
            #toDisplay = toDisplay.join(dayDfToPlot, how='right')

        # Display the values in a table
        display(toDisplay.round(3))

        # Plot la série temporelle + table en-dessous mais très long - pas nécessaire ici
        if False:
            fig, ax = plt.subplots()
            toDisplay = toDisplay.round(2)
            myplot = toDisplay.plot(ax=ax, table=True)
            myplot.axes.get_xaxis().set_visible(False)
            table = myplot.tables[0]
            # Setting the font size
            table.set_fontsize(24)        
            fig.tight_layout()

        # Event listener on the submit button: function to execute when clicked
        @CopyButton.on_click
        def react_on_click(b):
            CopyDataFrameToClipboard(toDisplay)
        
    
    

interactive(children=(Dropdown(description='Input file:', options=('données SWDE',), value='données SWDE'), Ou…

### Courbes de modulation - chaque jour de la semaine

In [67]:
# Choisir le fichier d'entrée à partir de la liste renseignée dans le tableur de paramètres
fileOptions = list(datasets.keys()) #.Nom.to_list()
selectFileWidget = widgets.Dropdown(
    options=fileOptions,
    value=fileOptions[0],
    description='Input file:',
    disabled=False,)

@interact
def SelectInputFile(f=selectFileWidget):
    data, dataNorm = PrepareDataframe(datasets[f])
    dataCols = [i for i in data.columns if i not in ['month_name', 'weekday_name', 'weekday']]

    selectStationWidget = widgets.RadioButtons(
        options=dataCols,
        value=dataCols[0],
        description='Station',
        layout=layout,
        disabled=False)
    
    ###### LAYOUT #######
    # Create elements to display
    out = widgets.Output()
    info = widgets.Label('Choose a station')
    CopyButton = CreateCopyButton('Copy to clipboard', 'success')

    # Arrange the elements in a grid
    grid_data = GridspecLayout(2, 5)
    grid_data[1,1] = info
    grid_data[1,3] = CopyButton
    grid_data[0,:] = out

    # Display
    display(grid_data)
    
    ###### INTERACTIONS #######
    @interact
    def SelectStation(station=selectStationWidget):
        plt.close('all')
        #stationData = data[[station, 'weekday', 'weekday_name']]
        stationDataNorm = dataNorm[[station, 'weekday', 'weekday_name']]

        # 1. Analyse sur les weekdays / weekends
        toDisplay = pd.DataFrame()
        for day in weekdays:
            dayDf = stationDataNorm[station][stationDataNorm['weekday_name']==day]
            dayDfToPlot = dayDf.groupby(dayDf.index.hour).median()
            toDisplay['{}_{}'.format(station, day)] = dayDfToPlot
            #dayDfToPlot.rename({station:'{}_{}'.format(station, day)}, inplace=True)
            #print(dayDfToPlot.columns)
            #toDisplay = toDisplay.join(dayDfToPlot, how='right')

        # Display the values in a table
        display(toDisplay.round(3))
        
        # Event listener on the submit button: function to execute when clicked
        @CopyButton.on_click
        def react_on_click(b):
            CopyDataFrameToClipboard(toDisplay)
        

interactive(children=(Dropdown(description='Input file:', options=('données SWDE',), value='données SWDE'), Ou…